# Exploratory Data Analysis (House Prices dataset) <a name = 'content'></a>    
  

### Steps   

* [Import](#import)
* [Data Loading](#loading)  
* [Mutual Information scores](#MI)      

# Import <a name = 'import'></a>

[Table of Contents](#content)

In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import mutual_info_regression

# Data Loading<a name = 'loading'></a>    

[Table of Contents](#content)  

In [121]:
file_path = '../../Data/train_raw.csv'

data = pd.read_csv(file_path, index_col = 'Id')

In [122]:
pd.set_option('display.max_columns', None)

data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


**Get potential features in one variable**

Devide all features into more and less potential according to my intuition analysis.    
*Read more: /Notebooks/Exploratory/intuition_features_analysis.xlsx*

In [123]:
potential = ['MSSubClass', 'MSZoning', 'LotArea', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 
             'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond',
             'Foundation', 'BsmtCond', 'BsmtFinType1', 'TotalBsmtSF', 'HeatingQC', 'CentralAir', 'Electrical', 'LowQualFinSF',
             'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Functional', 'GarageCars', 'GarageCond',
             'PoolQC', 'PoolArea', 'MiscVal']

non_potential = data[data.columns.difference(potential)]

# Mutual Information scores <a name = 'MI'></a>

[Table of Contents](#content)

**Steps**:    
1. [Missing Data processing](#missing_data)    
2. [Categorical Variables encoding](#cat_encoding)  
3. [Discrete and Continuous features](#disc_cont)   
4. [Mutual Information scores computing](#MI_computing)

I'm going to examine my intuition using mutual information metrics. Firstly I need to process missing values and encode categorical variables.

### Missing data<a name = 'missing_data'></a>

[Mutual Information](#MI)

In [124]:
X = data.copy()
y = X['SalePrice']
X = X[potential]


for col in X.columns:
    print(col, '- ', X[col].isnull().sum())

X.shape

MSSubClass -  0
MSZoning -  0
LotArea -  0
Utilities -  0
LotConfig -  0
Neighborhood -  0
Condition1 -  0
Condition2 -  0
HouseStyle -  0
OverallQual -  0
OverallCond -  0
YearBuilt -  0
YearRemodAdd -  0
ExterQual -  0
ExterCond -  0
Foundation -  0
BsmtCond -  37
BsmtFinType1 -  37
TotalBsmtSF -  0
HeatingQC -  0
CentralAir -  0
Electrical -  1
LowQualFinSF -  0
GrLivArea -  0
FullBath -  0
HalfBath -  0
BedroomAbvGr -  0
TotRmsAbvGrd -  0
Functional -  0
GarageCars -  0
GarageCond -  81
PoolQC -  1453
PoolArea -  0
MiscVal -  0


(1460, 34)

We don't want to lose 81 rows of data because of `GarageCond`, wich doesn't seem like mega-important variable, so we will drop this column. Missing values in `BsmtCond`, `BsmtFinType2` and `Electrical` are not a big part of the data, so we can just drop the rows. `PoolQC` seems like unusable variable, so we will drop the whole variable. Instead of it I will create a new feature `IsPool`, which will be equal to 1, if `PoolArea` > 0 (there is a pool) and 0, if `PoolArea` = 0 (there is no pool). Then I will drop `PoolArea` too.

In [125]:
X['IsPool'] = (X['PoolArea'] != 0).astype(int)

X = X.drop(['PoolQC', 'PoolArea', 'GarageCond'], axis = 1)
X = X.dropna(subset = ['BsmtCond', 'BsmtFinType1', 'Electrical'])

# Check that no missing values left
X.isnull().sum().max()

0

In [126]:
X.shape

(1422, 32)

We've lost about `2,6 %` of data. I can't say this is great news, but we definitely can live with that.

### Categorical Variables encoding  <a name = 'cat_encoding'></a>

[Mutual Information](#MI)

Our data isn't homogeneous, that'why we need to use different encodings depending on the content of the variables.   

Let's divide columns into three parts:

In [127]:
# For columns without hierarchy. (The dataset is too small to use the target encoding, so we will use OH-encoding even 
# on columns with relatively high cardinality. Again, dataset is small, so this approach won't slow down the model much)
dummies_encoding_cols = ['MSSubClass', 'MSZoning', 'Utilities', 'LotConfig',  'Foundation', 'Electrical', 'Condition1', 
                         'Condition2', 'HouseStyle', 'Neighborhood']

# For columns with hierarchy
ordinal_encoding_cols = ['ExterQual', 'ExterCond', 'BsmtCond', 'BsmtFinType1', 'HeatingQC', 'CentralAir', 'Functional']


**Get Dummies**

In [128]:
for col in X[dummies_encoding_cols]:
    X = pd.get_dummies(X, columns = [col], prefix = [col + '_'])

**Label Encoding**   

Since we need to keep hierarchy in the data, we will use `OrdinalEncoder` with custom order of encoding.

In [129]:
# ExterQual encoding
evaluation_order = [['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']]
evaluation_encoder = OrdinalEncoder(categories = evaluation_order)
X['ExterQual_encoded'] = evaluation_encoder.fit_transform(X[['ExterQual']])

# Seems like ExterCond, BsmtCond and HeatingQC have the same categories as ExterQual (see more in data_description.txt), 
# so we can use evaluation_encoder to encode them
X['ExterCond_encoded'] = evaluation_encoder.fit_transform(X[['ExterCond']]) 
X['BsmtCond_encoded'] = evaluation_encoder.fit_transform(X[['BsmtCond']]) 
X['HeatingQC_encoded'] = evaluation_encoder.fit_transform(X[['HeatingQC']]) 

# BsmtFinType1 encoding
BsmtFinType1_order = [['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']]
BsmtFinType1_encoder = OrdinalEncoder(categories = BsmtFinType1_order)
X['BsmtFinType1_encoded'] = BsmtFinType1_encoder.fit_transform(X[['BsmtFinType1']])

# CentralAir encoding
CentralAir_order = [['N', 'Y']]
CentralAir_encoder = OrdinalEncoder(categories = CentralAir_order)
X['CentralAir_encoded'] = CentralAir_encoder.fit_transform(X[['CentralAir']])

# Functional encoding
Functional_order = [['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ']]
Functional_encoder = OrdinalEncoder(categories = Functional_order)
X['Functional_encoded'] = Functional_encoder.fit_transform(X[['Functional']])

Since we have encoded columns now, we can drop original ones

In [130]:
X = X.drop(['ExterQual', 'ExterCond', 'BsmtCond', 'HeatingQC', 'BsmtFinType1', 'CentralAir', 'Functional'], axis = 1)

During missing data processing and categorical variables encoding we dropped a lot of data from `X`. Let's ensure that `y` fit `X`:

In [131]:
y = y[X.index]

X_y = X.index == y.index
np.any(~X_y) # Check if at least one index of y and X doesn't match

False

### Discrete and Continuing features <a name = 'disc_cont'></a>

[Mutual Information](#MI)   

We will use `mutual_info_regression` from scikit-learn, which is sensitive to the discrete/continuous variable property. So first we need to explicitly define where in our data is a variable of what type.

In [133]:
# Create separate variables for discrete and continuing features
continuing_features = ['LotArea', 'TotalBsmtSF', 'LowQualFinSF', 'GrLivArea', 'MiscVal']
discrete_features = set(X.columns) - set(continuing_features)
discrete_features = list(discrete_features)

Convert the data types to `inf` for discrete features and `float` for continuing features. This is also a requirement of mutual_info_regression.

In [134]:
X[continuing_features] = X[continuing_features].astype('float')
X[discrete_features] = X[discrete_features].astype('int')

discrete_bool_list = X.dtypes == int

### Mutual Information scores computing<a name = 'MI_computing'></a> 

[Mutual Information](#MI)

In [135]:
mi_scores = mutual_info_regression(X, y, discrete_features= discrete_bool_list)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [136]:
for feature in mi_scores.index:
    print(feature, ' ', "{:.3f}".format(mi_scores[feature]))

OverallQual   0.564
GrLivArea   0.487
YearBuilt   0.421
GarageCars   0.373
TotalBsmtSF   0.356
ExterQual_encoded   0.326
FullBath   0.286
YearRemodAdd   0.279
TotRmsAbvGrd   0.226
Foundation__PConc   0.181
HeatingQC_encoded   0.168
LotArea   0.161
BsmtFinType1_encoded   0.143
MSSubClass__60   0.116
OverallCond   0.112
Foundation__CBlock   0.109
MSZoning__RM   0.086
HalfBath   0.084
Neighborhood__NAmes   0.080
MSSubClass__30   0.079
BedroomAbvGr   0.077
Neighborhood__NridgHt   0.068
MSZoning__RL   0.062
CentralAir_encoded   0.058
Neighborhood__NoRidge   0.052
Electrical__SBrkr   0.049
Neighborhood__Somerst   0.046
Foundation__BrkTil   0.045
HouseStyle__2Story   0.043
Neighborhood__Gilbert   0.043
Neighborhood__CollgCr   0.039
Neighborhood__OldTown   0.037
Electrical__FuseA   0.034
MSZoning__FV   0.033
BsmtCond_encoded   0.031
Neighborhood__IDOTRR   0.028
MSSubClass__50   0.026
HouseStyle__1.5Fin   0.026
MSSubClass__160   0.025
ExterCond_encoded   0.024
LotConfig__CulDSac   0.024
Neighbo

In [141]:
mi_scores.shape

(109,)

In [147]:
selected_features = mi_scores[mi_scores > 0.05]
selected_features = list(selected_features.index)
selected_features

['OverallQual',
 'GrLivArea',
 'YearBuilt',
 'GarageCars',
 'TotalBsmtSF',
 'ExterQual_encoded',
 'FullBath',
 'YearRemodAdd',
 'TotRmsAbvGrd',
 'Foundation__PConc',
 'HeatingQC_encoded',
 'LotArea',
 'BsmtFinType1_encoded',
 'MSSubClass__60',
 'OverallCond',
 'Foundation__CBlock',
 'MSZoning__RM',
 'HalfBath',
 'Neighborhood__NAmes',
 'MSSubClass__30',
 'BedroomAbvGr',
 'Neighborhood__NridgHt',
 'MSZoning__RL',
 'CentralAir_encoded',
 'Neighborhood__NoRidge']

So now we have 25 features that are most strongly linked to the target. That's great!